In [1]:
#import libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from matplotlib import gridspec 

In [2]:
#load dataset
data = pd.read_csv("creditcard.csv") 

In [3]:
data.head(10) 

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [4]:
#describing the data
print(data.shape) 
print(data.describe()) 

(284807, 31)
                Time            V1            V2            V3            V4  \
count  284807.000000  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean    94813.859575  1.168375e-15  3.416908e-16 -1.379537e-15  2.074095e-15   
std     47488.145955  1.958696e+00  1.651309e+00  1.516255e+00  1.415869e+00   
min         0.000000 -5.640751e+01 -7.271573e+01 -4.832559e+01 -5.683171e+00   
25%     54201.500000 -9.203734e-01 -5.985499e-01 -8.903648e-01 -8.486401e-01   
50%     84692.000000  1.810880e-02  6.548556e-02  1.798463e-01 -1.984653e-02   
75%    139320.500000  1.315642e+00  8.037239e-01  1.027196e+00  7.433413e-01   
max    172792.000000  2.454930e+00  2.205773e+01  9.382558e+00  1.687534e+01   

                 V5            V6            V7            V8            V9  \
count  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean   9.604066e-16  1.487313e-15 -5.556467e-16  1.213481e-16 -2.406331e-15   
std    1.380247e+00  1.332271

In [5]:
#imbalance in the data
fraud = data[data['Class'] == 1] 
valid = data[data['Class'] == 0] 
outlierFraction = len(fraud)/float(len(valid)) 
print(outlierFraction) 
print('Fraud Cases: {}'.format(len(data[data['Class'] == 1]))) 
print('Valid Transactions: {}'.format(len(data[data['Class'] == 0]))) 

0.0017304750013189597
Fraud Cases: 492
Valid Transactions: 284315


In [6]:
#the amount details for fraudulent transaction
fraud.Amount.describe() 

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [7]:
#the amount details for normal transaction
valid.Amount.describe() 

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [8]:
#separating the X and the Y values
X = data.drop(['Class'], axis = 1) 
Y = data["Class"] 
print(X.shape) 
print(Y.shape) 
# getting just the values for the sake of processing  
# (its a numpy array with no columns) 
xData = X.values 
yData = Y.values 

(284807, 30)
(284807,)


In [9]:
#training and testing data bifurcation
from sklearn.model_selection import train_test_split 
#split the data into training and testing sets 
xTrain, xTest, yTrain, yTest = train_test_split(xData, yData, test_size = 0.2, random_state = 42) 

In [10]:
#building the Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier 
#random forest model creation 
rfc = RandomForestClassifier() 
rfc.fit(xTrain, yTrain) 
#predictions 
yPred = rfc.predict(xTest) 

In [11]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
acc = accuracy_score(yTest, yPred)
metrics = {
  'accuracy': accuracy_score(yTest, yPred),
  'precision': precision_score(yTest, yPred),
  'recall': recall_score(yTest, yPred),
  'f1score': f1_score(yTest, yPred)
}

with open('metrics.txt', 'w') as f:
  for key, value in metrics.items():
    f.write(f'{key}: {value}\n')

print("Metrics saved:", metrics)

Metrics saved: {'accuracy': 0.9995786664794073, 'precision': 0.9743589743589743, 'recall': 0.7755102040816326, 'f1score': 0.8636363636363636}


In [1]:
import pickle

# Get feature importances from the current model (rfc)
feature_importances = rfc.feature_importances_

# Get predictions for test set
predictions = rfc.predict(xTest)

# Create sharing package from current model
model_params = {
    'predictions': predictions,
    'feature_importances': feature_importances,
    'n_estimators': rfc.n_estimators,
    'max_features': rfc.max_features,
    'max_depth': rfc.max_depth if rfc.max_depth is not None else -1
}


with open('model_params.pkl', 'wb') as f:
  pickle.dump(model_params, f)


NameError: name 'rfc' is not defined

In [1]:
%pip install python-dotenv

import sys
sys.path.append('..')

from ipfs_configs import upload_file_to_ipfs

model_file_path = 'fraud-detection-model.ipynb'

print(f"Uploading file {model_file_path} to IPFS...")

model_ipfs_hash = upload_file_to_ipfs(model_file_path)

print(f"Uploaded file IPFS hash: {model_ipfs_hash}")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2


Uploading file fraud-detection-model.ipynb to IPFS...
Uploaded file IPFS hash: QmVjPrY2RB8t3N3wqgq1bUTk3SPweRU6ogC4JNEkH9A6JH


In [2]:
%pip install requests
import json
import requests


with open('metrics.txt', 'r') as file:
  metrics = file.read().strip()

# Convert metrics string to dictionary
metrics_dict = {}
for line in metrics.splitlines():
  if ': ' in line:
    key, value = line.split(': ', 1)
    metrics_dict[key] = int(float(value) * 100)

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: requests in d:\dml\dml_\dml\.venv\lib\site-packages (2.32.3)




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install cryptography

import sys
sys.path.append('..')

from ipfs_configs import upload_file_to_ipfs
from cryptography.fernet import Fernet

key = Fernet.generate_key()
cipher_suite = Fernet(key)

with open('model_params.pkl', 'rb') as file:
  original_data = file.read()

# Encrypt the data
encrypted_data = cipher_suite.encrypt(original_data)

# Write encrypted data to file
with open('model_params_encrypted.pkl', 'wb') as file:
  file.write(encrypted_data)

params_file_path = 'model_params_encrypted.pkl'

param_ipfs_hash = upload_file_to_ipfs(params_file_path)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [4]:
import base64

with open('model_hash', 'r') as file:
  ipfs_hash = file.read().strip()
# Create final JSON object
data = {
  'model_ipfs_hash': model_ipfs_hash,
  'metrics': metrics_dict,
  'param_ipfs_hash': param_ipfs_hash,
  'param_key': base64.b64encode(key).decode('utf-8')
}


# Convert to JSON string
json_data = json.dumps(data, indent=2)
print("JSON data:", data)

response = requests.post('http://127.0.0.1:5000/update-data', json=data)

JSON data: {'model_ipfs_hash': 'QmVjPrY2RB8t3N3wqgq1bUTk3SPweRU6ogC4JNEkH9A6JH', 'metrics': {'accuracy': 50, 'precision': 50, 'recall': 50, 'f1score': 50}, 'param_ipfs_hash': 'QmSd12ZuxVeMarAdiCLqNJjJGWegAq2wLK6o2UYVm26jyx', 'param_key': 'MW1seEppamV4Z25JRXc1QmR1V3ZvTUN0TWZDZFl4SElhMGFtMG9VMHJNND0='}


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /update-data (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000022DAA344650>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))